In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark = SparkSession.builder.appName('Filteration').getOrCreate()

23/10/20 21:08:52 WARN Utils: Your hostname, Sundeeps-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 11.38.254.110 instead (on interface en0)
23/10/20 21:08:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 21:08:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [32]:
spark

In [4]:
df = spark.read.csv("/Users/sundeepk/Desktop/Employees.csv", header=True, inferSchema = True)

In [5]:
df.show()

+------+----+-----------+------+
|  name| age|experience |salary|
+------+----+-----------+------+
|Krish |  31|         10|  3000|
| Sudha|  30|          8| 25000|
| sunny|  27|          3| 20000|
|  paul|  24|          1| 20000|
|Harsha|  21|          1| 15000|
|Shubam|  23|          2| 18000|
|mahesh|NULL|       NULL| 40000|
|  NULL|  34|         10|  NULL|
|  NULL|  23|       NULL|  NULL|
+------+----+-----------+------+



In [34]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience : integer (nullable = true)
 |-- salary: integer (nullable = true)



In [7]:
#Lets drop all the Null values 
df =df.na.drop()

In [8]:
#Lets perform the filter operation

df.filter("salary <=20000").show()

+------+---+-----------+------+
|  name|age|experience |salary|
+------+---+-----------+------+
|Krish | 31|         10|  3000|
| sunny| 27|          3| 20000|
|  paul| 24|          1| 20000|
|Harsha| 21|          1| 15000|
|Shubam| 23|          2| 18000|
+------+---+-----------+------+



In [9]:
#Just display your required columns 

df.filter("salary <=20000").select(['age', 'name']).show()

+---+------+
|age|  name|
+---+------+
| 31|Krish |
| 27| sunny|
| 24|  paul|
| 21|Harsha|
| 23|Shubam|
+---+------+



In [19]:
# lets put multiple conditions 

#AND function

df.filter((df["salary"] >15000) & (df["age"] <=27)).show()

+------+---+-----------+------+
|  name|age|experience |salary|
+------+---+-----------+------+
| sunny| 27|          3| 20000|
|  paul| 24|          1| 20000|
|Shubam| 23|          2| 18000|
+------+---+-----------+------+



In [28]:
#OR function 

df.filter(~(df["salary"] >= 15000) &
          (df["salary"] <= 20000) |
          (df["age"] <=23)).show()

+------+---+-----------+------+
|  name|age|experience |salary|
+------+---+-----------+------+
|Krish | 31|         10|  3000|
|Harsha| 21|          1| 15000|
|Shubam| 23|          2| 18000|
+------+---+-----------+------+



## ~ NOT OPERATION 
## | OR OPERATION 
## & AND OPERATION 


In [30]:
df_dpt = spark.read.csv('/Users/sundeepk/Desktop/departments.csv', header = True, inferSchema = True)

In [31]:
df_dpt.show()

+------+-------------+-------+
| Name |  Department |Salary |
+------+-------------+-------+
|Krish |data science |  10000|
|Krish |          IOT|   5000|
|Mahesh|    Big data |   4000|
|Krish |    Big data |   4000|
|Mahesh|data science |   3000|
|Sudha |         IOT |  20000|
|sudha |    Big data |   5000|
|sunny |data science |  10000|
|sunny |    Big data |   2000|
+------+-------------+-------+



In [33]:
df_dpt.printSchema()

root
 |-- Name : string (nullable = true)
 |-- Department : string (nullable = true)
 |-- Salary : integer (nullable = true)



In [39]:
#Lets see who gets the max salary out of everyone

#First lets group the names and then perform .sum()

df_dpt.groupBy('Name ').sum().show()

+------+------------+
| Name |sum(Salary )|
+------+------------+
|Sudha |       20000|
|Krish |       19000|
|sudha |        5000|
|sunny |       12000|
|Mahesh|        7000|
+------+------------+



In [44]:
## Lets find which department gives max salary 


df_dpt.groupBy('Department ').max().show()

+-------------+------------+
|  Department |max(Salary )|
+-------------+------------+
|          IOT|        5000|
|         IOT |       20000|
|    Big data |        5000|
|data science |       10000|
+-------------+------------+



In [46]:
## How to count the people in a department 

df_dpt.groupBy('Department ').count().show()

+-------------+-----+
|  Department |count|
+-------------+-----+
|          IOT|    1|
|         IOT |    1|
|    Big data |    4|
|data science |    3|
+-------------+-----+



In [49]:
#WE CAN ALSO DO AGG function 

df_dpt.agg({'Salary ':'sum'}).show()

+------------+
|sum(Salary )|
+------------+
|       63000|
+------------+

